# Perfect indistinguishability

Caratterizzazione alternativa (ma equivalente) della perfect secrecy.


## Esperimento di indistinguishability

PrivK<sup>eav</sup><sub>M,π</sub>

PrivK $\rightarrow$ crittografia chiave privata \
π $\rightarrow$ nome del cifrario $\rightarrow$ π = (Gen, E, D) \
eav $\rightarrow$ eavesdropper $\rightarrow$ attaccante più debole (può soltanto guardare il canale pubblico)\
M $\rightarrow$ Mallory, attaccante

1. M $\rightarrow$ A : x<sub>0</sub>, x<sub>1</sub> &emsp; <i>attaccante M invia due plaintext ad A</i>
2. A : k $\leftarrow$ Gen() &emsp; <i>A genera una chiave con l'algoritmo Gen() $\in$ π</i> \
    &emsp; b $\leftarrow$ {0, 1} &emsp; <i>A genera un bit <b>b</b> uniformemente random</i>
3. A $\rightarrow$ M : y $\leftarrow$ E<sub>k</sub>(x<sub>b</sub>) &emsp; <i>A invia ad M un ciphertext ottenuto da uno dei 2 plaintext (scelto in base a <b>b</b>)</i>
4. M : b' $\leftarrow$ ... &emsp; <i>M vuole calcolare <b>b'</b> in modo che <b>b'=b</b> (vuole indovinare <b>b</b>)</i>

L'esperimento ha successo (PrivK<sup>eav</sup><sub>M,π</sub> = 1) se <b>b'=b</b>

### Cifrario perfectly indistinguishable

π è <b>perfectly indistuinguishable</b> (e anche perfetto) se:

$\forall$ M : Pr(PrivK<sup>eav</sup><sub>M,π</sub> = 1) = 1/2 &emsp; <i><b>per ogni avversario</b> M , <b>la probabilità che l'esperimento abbia successo è 1/2</b> (anche il migliore avversario è costretto a tirare a indovinare come sua migliore strategia)</i>

Qualunque M vuol dire anche l'avversario con HW illimitato e tempo illimitato (irrealistico)

### Esercizio 

P = C = Z<sup>2</sup><sub>26</sub> &emsp; <i>due caratteri in Z<sub>26</sub></i>\
K = Z<sub>26</sub> &emsp;<i>chiave con un singolo carattere</i>\
Gen() : {k $\leftarrow$ K} &emsp; <i>chiave uniformemente random</i>

E<sub>k</sub> (x x') = (x + k)(x' + k) &emsp; <i>la stessa chiave viene utilizzata per cifrare caratteri diversi $\rightarrow$ sicuramente non è un cifrario perfetto</i>

1. M $\rightarrow$ A : x<sub>0</sub>, x<sub>1</sub> &emsp; x<sub>0</sub> = 00 &emsp; x<sub>1</sub> = 01 &emsp; <i>M invia questi due messaggi ad A</i>
2. A : k $\leftarrow$ Gen() &emsp; b$\leftarrow$ {0, 1} &emsp; <i>A genera il bit random <b>b</b></i>
3. A $\rightarrow$ M : y $\leftarrow$ E<sub>k</sub>(x<sub>b</sub>) &emsp; y = zz' &emsp; <i>A fa l'encryption di un messaggio (scelto in base a <b>b</b>) che è composto da due caratteri z e z'</i>
4. M : b' $\leftarrow$ ... &emsp;<i>M deve indovinare <b>b</b></i>

b' = {0 se z=z' ; 1 altrimenti} &emsp;<i>la sua strategia è: dire che è il messaggio 0 (caratteri uguali nel plaintext) se nel ciphertext ci sono i caratteri uguali, 1 altrimenti </i>

Pr(PrivK<sup>eav</sup><sub>M,π</sub> = 1) = 1 &emsp; <i>con questa strategia, la probabilità di indovinare è 1 $\rightarrow$ indovina sempre (passando quei messaggi)</i>

Pr(PrivK<sup>eav</sup><sub>M,π</sub> = 1) &emsp;=&emsp; Pr(b'=0, b=0) + Pr(b'=1, b=1) &emsp;=&emsp; Pr(b'=0 | b=0) Pr(b=0) + Pr(b'=1 | b=1) Pr(b=1)

= 1/2 Pr(b'=0 | b=0) + 1/2 Pr(b'=1 | b=1) &emsp;=&emsp; 1/2 1 + 1/2 1 = 1 $\neq$ 1/2 $\rightarrow$ il cifrario <b>non è</b> perfectly indistinguishable!

# Experiments

<b>IMPORTANT:</b> The following code is based from this: https://github.com/bitbart/crypto-pub/

### Abstract:

In [2]:
import secrets
from abc import ABC, abstractmethod

def int_of_chr(n):
    return ord(n)-ord('a')

def chr_of_int(n):
    return chr(n + ord('a'))

# Abstract class:

class Cipher(ABC):

    @abstractmethod
    def gen(self,n):
        pass

    @abstractmethod
    def enc(self,x,k):
        pass

    @abstractmethod
    def dec(self,y,k):
        pass

    @abstractmethod
    def string_of_key(self,k):
        pass

    @abstractmethod
    def key_of_string(self,s):
        pass
    
    # Alice's operations in the perfect indistinguishability experiment:
    def alice(self,x0,x1, print_key=True):
        k = self.gen()                         # generate key
        
        if(print_key):
            print("k = " + ''.join(str(k)))
       
        b = secrets.choice([0,1])              # generate random bit
        y = self.enc(x1 if b else x0,k)        # encrypt xb = x0 if b=0, x1 if b=1
        return (b,y)

    
class Mallory(ABC):
    
    @abstractmethod
    def plaintexts(): # the plaintexts the adversary chooses
        pass
    
    @abstractmethod
    def guess(y):     # Mallory's strategy guessing the bit
        pass
    
    
class Experiment():
    
    def __init__(self, pi, m, n):
        self.pi = pi # cipher
        self.m = m   # Mallory
        self.n = n   # total number of experiments
        self.s = 0   # number of experiments where Mallory was the winner
        
    def run(self, print_steps=True):
        for i in range(self.n): 
            if(print_steps):
                print("\n" + "Experiment #" + str(i+1))

             # M -> A : x0, x1
            (x0,x1) = self.m.plaintexts()
            
            if(print_steps):
                print("x0 = " + x0)
                print("x1 = " + x1)

             # A -> M : y = Ek(x[b])
            (b,y) = self.pi.alice(x0,x1, print_key=print_steps) # Alice generates the key, the random bit and performs the encryption
           
            if(print_steps):
                print("b = " + str(b))   
                print("y = " + y)

             # M : bm   
            bm = self.m.guess(y)
            
            if(print_steps):
                print("bm = " + str(bm))

            if bm==b:
                if(print_steps):
                    print("PrivK = 1 (Mallory wins)")
                self.s = self.s+1
            else:
                if(print_steps):
                    print("PrivK = 0 (Mallory loses)")
        
        print("\n" + "Percentage of success: " + str(self.s*100./self.n))

### Uncipher:

In [3]:
class Uncipher(Cipher):

    def gen(self):
        k = secrets.randbelow(26)
        return k

    def enc(self,x,k):
        return x

    def dec(self,y,k):
        return y

    def string_of_key(self,k):
        return chr_of_int(k)

    def key_of_string(self,s):
        return int_of_chr(s)

In [4]:
# Adversary for the Uncipher:

class MalloryUncipher(Mallory):
    
    def plaintexts(self):
        return ("a","b")

    def guess(self,y):
        if y=='a':
            bm = 0
        else:
            bm = 1

        return bm

In [5]:
# Privk-eav experiment with the Uncipher:

pi = Uncipher()
m = MalloryUncipher()

experiment = Experiment(pi, m, 100) # 100 experiments

experiment.run(print_steps=False)


Percentage of success: 100.0


### Shift cipher in ECB-mode (with uniform keys and plaintexts of arbitrary length):

In [6]:
# Similar to the one defined in 'shift.ipynb'

class ShiftECB(Cipher):

    def gen(self):
        k = secrets.randbelow(26)
        return k

    def enc(self,x,k):
        # Ek(x1 x2 x3 ... xn,k) = (x1+k)%26 (x2+k)%26 (x3+k)%26 ... (xn+k)%26
        return  ''.join(map(lambda n : chr_of_int((int_of_chr(n) + k)%26), x))

    def dec(self,y,k):
        return  ''.join(map(lambda n : chr_of_int((int_of_chr(n) - k)%26), y))

    def string_of_key(self,k):
        return chr_of_int(k)

    def key_of_string(self,s):
        return int_of_chr(s)

In [7]:
# Adversary for the ShiftECB:

class MalloryShiftECB(Mallory):
    
    def plaintexts(self):
        return ("aa","ab")

    def guess(self,y):
        if y[0]==y[1]:
            bm = 0
        else:
            bm = 1
        return bm

In [8]:
# Privk-eav experiment with the ShiftECB:

pi = ShiftECB()
m = MalloryShiftECB()

experiment = Experiment(pi, m, 100) # 100 experiments

experiment.run(print_steps=False)


Percentage of success: 100.0


### Shift cipher with non-uniform keys and plaintexts of length 1:

In [9]:
class Shift1Unbal(Cipher):

    def gen(self):
        a = secrets.choice([0,1])
        
        if a==0:
            k=25
        else:
            k = secrets.randbelow(25)
            
        return k

    def enc(self,x,k):
        assert(len(x)==1),"Plaintext must have length 1"
        return  ''.join(map(lambda n : chr_of_int((int_of_chr(n) + k)%26), x))

    def dec(self,y,k):
        assert(len(x)==1),"Ciphertext must have length 1"    
        return  ''.join(map(lambda n : chr_of_int((int_of_chr(n) - k)%26), y))

    def string_of_key(self,k):
        return chr_of_int(k)

    def key_of_string(self,s):
        return int_of_chr(s)

In [10]:
# Adversary for the Shift1Unbal:

class MalloryShift1Unbal(Mallory):
    
    def plaintexts(self):
        return ('a','b')
    
    def guess(self,y):
        if y=='z':   # if y=='z', M says the original message *probably* was 'a' (the message 0) because 25 is the most probable key
            bm = 0
        else:
            bm = 1
            
        return bm

In [11]:
# Privk-eav experiment with the ShiftECB:

pi = Shift1Unbal()
m = MalloryShift1Unbal()

experiment = Experiment(pi, m, 100) # 100 experiments

experiment.run(print_steps=False)


Percentage of success: 71.0


L'avversario non indovina sempre (100%) ma la probabilità che l'esperimento abbia successo è intorno al 75% che è comunque maggiore del 50% come richiede la definizione. \
Quindi il cifrario Shift1Unbal (Shift cipher su singolo carattere, ma con <b>chiavi non uniformi</b>) <b>non è perfectly indistinguishable</b> (e quindi neanche perfetto).

## Shift cipher in OTP-mode for the first n chars, then uncipher

In [12]:
class ShiftLazyOTP(Cipher):
    
    def __init__(self, n):
        assert(n>0),"n must be greater than 0"
        self.n = n

    def gen(self):  
        k = []
        for i in range(self.n): # a truly random key but for the first n characters
            k.append(secrets.randbelow(26))
        return k

    def enc(self,x,k):
        d = len(x) - len(k)
        if d>0:   # padding
            k = k + [0] * d
        y = []
        for i in range(len(x)):
            y.append(chr_of_int((int_of_chr(x[i]) + k[i])%26))
        y = ''.join(y)
        return y
   
    def dec(self,y,k):
        d = len(y) - len(k)
        if d>0:   # padding
            k = k + [0] * d
        x = []
        for i in range(len(y)):
            x.append(chr_of_int((int_of_chr(y[i]) - k[i])%26))
        x = ''.join(x)
        return x

    def string_of_key(self,k):
        s = ''.join(map (lambda ki : chr_of_int(ki), k))
        return s

    def key_of_string(self,s):
        # from string to list of int
        l = list(filter(lambda i : (i in s and i!="\n"), s))
        k = list(map(lambda ki : int_of_chr(ki), l))
        return k

In [13]:
# Adversary for the ShiftLazyOTP:

class MalloryShiftLazyOTP(Mallory):
    
    def plaintexts(self):
        return ("aaaaaa","aaaaab")
    
    def guess(self,y):
        if y[5]=="a": # M knows that after n characters, the encryption performs a Uncipher (Kerckhoffs' assumption) 
            bm = 0
        else:
            bm = 1
            
        return bm

In [14]:
# Privk-eav experiment with the ShiftLazyOTP:

pi = ShiftLazyOTP(5)
m = MalloryShiftLazyOTP()

experiment = Experiment(pi, m, 100) # 100 experiments

experiment.run(print_steps=False)


Percentage of success: 100.0


M con questi due plaintext indovina sempre, perchè sa <b>(per il principio di Kerckhoffs)</b> che dopo n caratteri l'encryption non lavora più in OTP-mode, ma dall'(n+1)-esimo carattere, lascia i successivi così come sono anche nel ciphertext (Uncipher): questo è il motivo per il quale nella sua guess(), l'avversario controlla il 6 carattere (in posizione 5): se è una a, come nel primo messaggio, allora il messaggio cifrato è proprio il primo; altrimenti è il messaggio con indice 1.

## Vigenere cipher with non-uniform keys and plaintexts of length 2

In [15]:
class Vigenere2Unbal(Cipher):
    
    def gen(self):
        a = secrets.choice([0,1])
        k0 = secrets.randbelow(26)      # first key's character truly random
        if a==0:
            k1 = k0                     # 50% chance of the second character being equal to the first one
        else:
            k1 = secrets.randbelow(26)  # 50% chance of the second character being truly random as the first one (OTP)
        return [k0,k1]

    def enc(self,x,k):
        assert(len(x)==2 and len(k)==2)
        y0 = ''.join(map(lambda n : chr_of_int((int_of_chr(n) + k[0])%26), x[0]))  # first key's character
        y1 = ''.join(map(lambda n : chr_of_int((int_of_chr(n) + k[1])%26), x[1]))  # second key's character
        return y0+y1

    def dec(self,y,k):
        assert(len(y)==2 and len(k)==2)
        x0 = ''.join(map(lambda n : chr_of_int((int_of_chr(n) - k[0])%26), y[0]))
        x1 = ''.join(map(lambda n : chr_of_int((int_of_chr(n) - k[1])%26), y[1]))
        return x0+x1


    def string_of_key(self,k):
        s = ''.join(map (lambda ki : chr_of_int(ki), k))
        return s

    def key_of_string(self,s):
        # from string to list of int
        l = list(filter(lambda i : (i in s and i!="\n"), s))
        k = list(map(lambda ki : int_of_chr(ki), l))
        return k

In [16]:
# Adversary for the Vigenere2Unbal:

class MalloryVigenere2Unbal(Mallory):
    
    def plaintexts(self):
        return ("aa","ab")
    
    def guess(self,y):
        if y[0]==y[1]: 
            bm = 0
        else:
            bm = 1
            
        return bm

In [17]:
# Privk-eav experiment with the Vigenere2Unbal:

pi = Vigenere2Unbal()
m = MalloryVigenere2Unbal()

experiment = Experiment(pi, m, 100) # 100 experiments

experiment.run(print_steps=False)


Percentage of success: 75.0


Distribuzione delle chiavi non uniforme $\rightarrow$ <b>non indistinguibile e quindi non perfetto</b>

## OTP

In [18]:
from functools import reduce

class OTP(Cipher):

    def __init__(self, n):
        self.n = n

    def gen(self):
        k = []
        for i in range(self.n):
            k.append(secrets.choice([0,1]))
        return k

    def enc(self,x,k):
        assert (len(x)==self.n), "Plaintext must have length " + str(self.n)
        assert (len(x)==len(k)), "Plaintexts and key have different lengths"
        assert (reduce(lambda z, b: z and b in ['0','1'], x, True)), "Plaintext not bitstring"

        y = reduce(lambda s, z : s + z, map(lambda xi, ki : str(int(xi) ^ ki), x, k), "")
        return y

    def dec(self,y,k):
        assert (len(y)==self.n), "Ciphertext must have length " + str(self.n)
        assert (len(y)==len(k)), "Ciphertext and key have different lengths"
        assert (reduce(lambda z, b: z and b in ['0','1'], y, True)), "Ciphertext not bitstring"

        x = reduce(lambda s, z : s + z, map(lambda yi, ki : str(int(yi) ^ ki), y, k), "") # concatenates the elements returned by the map (i.e. the XOR)
        return x

    def string_of_key(self,k):
        s = ''.join(map (lambda ki : str(ki), k))
        return s

    def key_of_string(self,s):
        # from string to list of chr
        l = list(filter(lambda i : (i in s and i!="\n"), s))
        # from list of chr to list of int
        k = list(map(lambda ki : int(ki), l))
        return k

In [19]:
import random

class MalloryOTP(Mallory):
    # An adversary that randomly says 0 or 1
    
    def plaintexts(self):
        return ("0000000","0101010")   # they don't matter against OTP: M can only randomly guess

    def guess(self,y):
        return random.randint(0,1)

In [20]:
# Privk-eav experiment with the OTP:

pi = OTP(7)
m = MalloryOTP()

experiment = Experiment(pi, m, 100000) # a lot of experiments

experiment.run(print_steps=False)


Percentage of success: 49.797


As we increase the number of experiments, the probability tends to 50%

## OTPlastXOR: OTP where the last bit of the key if the XOR of the previous bits

In [30]:
class OTPlastXor(OTP):

    def __init__(self, n):
        assert(n>0),"n must be greater than 0"
        self.n = n

    def gen(self):
        k = []
        for i in range(self.n-1):
            k.append(secrets.choice([0,1]))

        lb = reduce(lambda z, y: z ^ y, k, 0)
        k.append(lb)
        return k
    
    # The only overwrite we want to do is on the key (the gen method). Since this cipher inherits from the OTP the other methods are the same from the upper class

In [36]:
# Adversary for OTPlastXor

class MalloryOTPlastXor(Mallory):
    
    def plaintexts(self):
        return ("000","001")

    def guess(self,y):
        n = 0
        for bi in y[:-1]:   # (Excluding the last bit)
            n = n ^ int(bi) # The XOR of the previous bits in y
            
        if n==int(y[-1]):  # If the XOR of the previous bits is equal to the last bit of the ciphertext
            bm = 0
        else:
            bm = 1
        return bm

In [37]:
# Privk-eav experiment with the OTPlastXor:

pi = OTPlastXor(3)
m = MalloryOTPlastXor()

experiment = Experiment(pi, m, 100)

experiment.run(print_steps=False)


Percentage of success: 100.0


L'avversario vince sempre: l'algoritmo gen() rivela troppe informazioni sul keystream: l'avversario infatti si limita a replicare la strategia sul ciphertext (controlla lo XOR dei bit precedenti all'ultimo: questo è in comune con la chiave: quello che cambia è l'ultimo bit che non è stato generato in maniera casuale, infatti è lo XOR del keystream, e mettendo nei plaintext 0 e 1 come ultimo bit può portare avanti la strategia vincente!).\
Ovviamente, questo cifrario non è <b>perfectly indistinguishable</b> anzi è proprio inutilizzabile.

## TwoTP: OTP where the last half of the key is the same as the first one (this one is obtained as OTP) 

In [38]:
class TwoTP(OTP):

    def __init__(self, n):
        assert(n%2==0),"TwoTP: n must be even"
        self.n = n

    def gen(self):
        k = []
        
        for i in range(int(self.n/2)):
            k.append(secrets.choice([0,1]))
        
        k = k + k    # !!!
        
        return k

In [67]:
# Adversary for TwoTP

class MalloryTwoTP(Mallory):
    
    def plaintexts(self):
        return ("0000","0010") # 0000 and 0011 works as well

    def guess(self,y):
        if y[0]==y[int(len(y)/2)]:   # Compares the first bit of the first part of the key and the first bit of the last part 
            bm = 0
        else:
            bm = 1
        return bm

In [72]:
# Adversary for TwoTP

class MalloryTwoTP(Mallory):
    
    def plaintexts(self):
        return ("0000","0011")

    def guess(self,y):
        if y[0]==y[int(len(y)/2)]:
            bm = 0
        else:
            bm = 1
        return bm

In [73]:
# Privk-eav experiment with the TwoTP:

pi = TwoTP(4)
m = MalloryTwoTP()

experiment = Experiment(pi, m, 100)

experiment.run(print_steps=False)


Percentage of success: 100.0


Anche qui l'avversario vince sempre, rendendo questo cifrario assolutamente <b>non perfectly indistinguishable</b>.\
La strategia dell'avversario è mettere a 1 il bit a metà nei plaintext: dal momento che la chiave è ripetuta una seconda volta, la seconda metà di chiave che va a crifrare la seconda parte del ciphertext (dove appare 1 come terza posizione) renderà il corrispondente bit del ciphertext diverso dal primo bit.\
Altrimenti, se sono uguali, il messaggio è il primo, quello con tutti 0.

## Quasi-OTP: OTP ma non ammette la chiave dove tutti i bit sono 0

In [74]:
class QuasiOTP(OTP):

    def __init__(self, n):
        self.n = n

    def gen(self):
        found = False
        while not found:
            k = []
            for i in range(self.n):
                k.append(secrets.choice([0,1]))
            for bi in k:
                if bi==1:
                    found=True
        return k

In [82]:
# Adversary for QuasiOTP

class MalloryQuasiOTP(Mallory):
    
    def plaintexts(self):
        return ("0000","1111")

    def guess(self,y):
        allZero = True          # allZero=True iff all bits in ciphertext are 0
        for yi in y:
            if int(yi)==1:
                allZero = False

        if allZero:
            bm = 1
        else:
            bm = 0
            
        return bm

In [89]:
# Privk-eav experiment with the Quasi-OTP:

pi = QuasiOTP(4)
m = MalloryQuasiOTP()

experiment = Experiment(pi, m, 1000000)

experiment.run(print_steps=False)


Percentage of success: 53.2861


A prima vista sembrerebbe essere perfectly indistinguishable, <b>TUTTAVIA</b>, aumentando sensibilmente il numero di esperimenti, il cifrario sembra superare la soglia del 50%.\
Infatti, lo spazio delle chiavi è minore dello spazio dei plaintext (0000 non appare nelle chiavi, ma può apparire nel plaintext: l'avversario sta sfruttando questa possibilità per avere un qualche margine di probabilità di vincita in più. Quindi il Quasi-OTP <b>non è perfectly indistinguishable</b>.